In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import os
import sys

(parent_folder_path, current_dir) = os.path.split(os.path.abspath(''))
sys.path.append(parent_folder_path)

os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"

import os
# import pickle
import numpy as np
from contextlib import nullcontext
import torch

from equities.model import GPTConfig, GPT
from equities.data_processing import itch_encoding

In [56]:
# INIT PARAMS
# -----------------------------------------------------------------------------
init_from = 'resume' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
out_dir = parent_folder_path + '/out' # ignored if init_from is not 'resume'
dataset = '12302019.NASDAQ_ITCH50_AAPL_message_proc.npy' # dataset to use for initial prompt
# start = "\n" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
num_context_msgs = 3 # number of messages from dataset to use as context
# num_samples = 10 # number of samples to draw
num_samples = 1 # number of samples to draw
# max_new_tokens = 500 # number of tokens generated in each sample
max_new_tokens = 1 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 42
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster
# exec(open('equities/configurator.py').read()) # overrides from command line or config file
# -----------------------------------------------------------------------------

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)


In [4]:
# model
if init_from == 'resume':
    # init from a model saved in a specific directory
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    gptconf = GPTConfig(**checkpoint['model_args'])
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)

model.eval()
model.to(device)

number of parameters: 94.57M


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(12515, 768)
    (wpe): Embedding(10367, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=False)
          (c_proj): Linear(in_features=768, out_features=768, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=3072, out_features=768, bias=False)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=768, out_features=12515, bias=False)
)

In [57]:
# define path for sample data
# dataset = '12302019.NASDAQ_ITCH50_AAPL_message_proc.npy'
data_dir = os.path.join('dataset/proc/ITCH/test/', dataset)
data_dir = parent_folder_path + '/' + data_dir

# grab sample data to use as context
context_dataset = np.load(data_dir, mmap_mode='r')
X_raw = np.array(context_dataset[0:num_context_msgs])
print("X_raw.shape:", X_raw.shape)
print("X_raw:", X_raw)

# encode the sample data
vocab = itch_encoding.Vocab()
X = itch_encoding.encode_msgs(X_raw, vocab.ENCODING)
print("X.shape:", X.shape)
print("X:", X)

# decode the sample data (will be missing order id, price_abs, old_id, and old_price_abs)
print("decoded X:", itch_encoding.decode_msgs(X, vocab.ENCODING))
print([ "ticker", "NA_VAL",
        "event_type", "direction", "NA_VAL", "price", "fill_size", "remain_size",
        "delta_t_s", "delta_t_ns", "time_s", "time_ns",
        "NA_VAL", "price_ref", "fill_size_ref", "time_s_ref", "time_ns_ref", "NA_VAL"])

X_raw.shape: (3, 18)
X_raw: [[      40  7872401        1        0    27499     -999      100    -9999
         0 15161946    34200 22603012    -9999    -9999    -9999    -9999
     -9999    -9999]
 [      40  7872405        1        0    28368     -595      100    -9999
         0     6899    34200 22609911    -9999    -9999    -9999    -9999
     -9999    -9999]
 [      40  7872421        1        1    30400      999      100    -9999
         0    16782    34200 22626693    -9999    -9999    -9999    -9999
     -9999    -9999]]
X.shape: (3, 24)
X: [[12051  1003 12010 12008 12007  1108     2     3    18   164   949    37
    203    25   606    15     2     2     2     2     2     2     2     2]
 [12051  1003 12010 12008 11603  1108     2     3     3     9   902    37
    203    25   612   914     2     2     2     2     2     2     2     2]
 [12051  1003 12011 12009 12007  1108     2     3     3    19   785    37
    203    25   629   696     2     2     2     2     2     2     2     

In [28]:
encoded_tok_len = X.shape[1]
print("encoded_tok_len:", encoded_tok_len)

encoded_tok_len: 24


In [40]:
# prepare context tensor
x = (torch.tensor(X.reshape(-1), dtype=torch.long, device=device)[None, ...])
print("x.shape:", x.shape)
print("x:", x)

# run generation
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            # y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            y = model.generate(x, max_new_tokens*encoded_tok_len, temperature=temperature, top_k=top_k)
            # print(decode(y[0].tolist()))
            print("last generated msg:", y[0][-1].tolist())
            # print(y[0].tolist())
            print('---------------')
        print("new sequence", y[0].tolist())

x.shape: torch.Size([1, 72])
x: tensor([[12051,  1003, 12010, 12008, 12007,  1108,     2,     3,    18,   164,
           949,    37,   203,    25,   606,    15,     2,     2,     2,     2,
             2,     2,     2,     2, 12051,  1003, 12010, 12008, 11603,  1108,
             2,     3,     3,     9,   902,    37,   203,    25,   612,   914,
             2,     2,     2,     2,     2,     2,     2,     2, 12051,  1003,
         12011, 12009, 12007,  1108,     2,     3,     3,    19,   785,    37,
           203,    25,   629,   696,     2,     2,     2,     2,     2,     2,
             2,     2]], device='cuda:0')
generated token: 2
---------------
new sequence [12051, 1003, 12010, 12008, 12007, 1108, 2, 3, 18, 164, 949, 37, 203, 25, 606, 15, 2, 2, 2, 2, 2, 2, 2, 2, 12051, 1003, 12010, 12008, 11603, 1108, 2, 3, 3, 9, 902, 37, 203, 25, 612, 914, 2, 2, 2, 2, 2, 2, 2, 2, 12051, 1003, 12011, 12009, 12007, 1108, 2, 3, 3, 19, 785, 37, 203, 25, 629, 696, 2, 2, 2, 2, 2, 2, 2, 2, 12051, 10

In [52]:
# print the last message in the generated sequence
print("last generated msg:", y[0][-24:].tolist())

# print(y[0].tolist())
print("y:", y)

# decode the generated sequence
# print("decoded y:", itch_encoding.decode_msg(y[0][-24:].tolist(), vocab.ENCODING))
print("decoded msg:", itch_encoding.decode_msg(np.array(y[0][-24:].tolist()), vocab.ENCODING))
print([ "ticker", "NA_VAL",
        "event_type", "direction", "NA_VAL", "price", "fill_size", "remain_size",
        "delta_t_s", "delta_t_ns", "time_s", "time_ns",
        "NA_VAL", "price_ref", "fill_size_ref", "time_s_ref", "time_ns_ref", "NA_VAL"])


last generated msg: [12051, 1003, 12010, 12008, 11016, 1108, 2, 3, 3, 3, 3, 37, 584, 23, 550, 755, 2, 2, 2, 2, 2, 2, 2, 2]
y: tensor([[12051,  1003, 12010, 12008, 12007,  1108,     2,     3,    18,   164,
           949,    37,   203,    25,   606,    15,     2,     2,     2,     2,
             2,     2,     2,     2, 12051,  1003, 12010, 12008, 11603,  1108,
             2,     3,     3,     9,   902,    37,   203,    25,   612,   914,
             2,     2,     2,     2,     2,     2,     2,     2, 12051,  1003,
         12011, 12009, 12007,  1108,     2,     3,     3,    19,   785,    37,
           203,    25,   629,   696,     2,     2,     2,     2,     2,     2,
             2,     2, 12051,  1003, 12010, 12008, 11016,  1108,     2,     3,
             3,     3,     3,    37,   584,    23,   550,   755,     2,     2,
             2,     2,     2,     2,     2,     2]], device='cuda:0')
decoded msg: [      40    -9999        1        0    -9999       -8      100    -9999
       

In [62]:
X_true = np.array(context_dataset[0:num_context_msgs+max_new_tokens])
print("X_true:", X_true)

print("true last msg:", X_true[-1])

X_true: [[      40  7872401        1        0    27499     -999      100    -9999
         0 15161946    34200 22603012    -9999    -9999    -9999    -9999
     -9999    -9999]
 [      40  7872405        1        0    28368     -595      100    -9999
         0     6899    34200 22609911    -9999    -9999    -9999    -9999
     -9999    -9999]
 [      40  7872421        1        1    30400      999      100    -9999
         0    16782    34200 22626693    -9999    -9999    -9999    -9999
     -9999    -9999]
 [      40  7872457        1        1    29532      569      100    -9999
         0    10618    34200 22637311    -9999    -9999    -9999    -9999
     -9999    -9999]]
true last msg: [      40  7872457        1        1    29532      569      100    -9999
        0    10618    34200 22637311    -9999    -9999    -9999    -9999
    -9999    -9999]
